In [1]:
import seaborn as sns
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

import env
import wrangle
import split_scale
import evaluate

### Using the data on student grades from this lesson, complete the following:

In [2]:
pd.read_csv('student_grades.csv')  

,student_id,exam1,exam2,exam3,final_grade
0,1,100.0,90,95,96
1,2,98.0,93,96,95
2,3,85.0,83,87,87
3,4,83.0,80,86,85
4,5,93.0,90,96,97
...,...,...,...,...,...
99,100,70.0,65,78,77
100,101,62.0,70,79,70
101,102,58.0,65,70,68
102,103,57.0,65,75,65


- Split the data into train and test datasets.

- Create a model that uses exam 1 to predict the final grade.

- Create a model that uses exam 2 to predict the final grade.

- Compare your models in the following manner:

> Calculate the mean squared error

> Visualize the residuals. Create a seperate visualization for each model.

> Visualize the actual vs the predicted values. Create a seperate visualization for each model.

> Bonus: Combine the seperate visualizations for each model into a single visualization. Is this visual helpful?

- Create a model that uses exam 1 and exam 3 to predict final grade. How does this model compare to your previous ones?

- Take your best preforming model and measure its performance on the test data set. How does performance differ between train and test?